In [43]:
!pip install emoji


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import numpy as np
import pandas as pd

In [45]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

In [46]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [47]:
import emoji as emoji

In [48]:
emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [49]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [50]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [51]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [52]:
from keras.utils import to_categorical

In [53]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [54]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [55]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [56]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

C:\Users\abhis\AppData\Local\Temp\ipykernel_25632\2176504226.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()
C:\Users\abhis\AppData\Local\Temp\ipykernel_25632\2176504226.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()
C:\Users\abhis\AppData\Local\Temp\ipykernel_25632\2176504226.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()
C:\Users\abhis\AppData\Local\Temp\

In [58]:
print(emb_XT.shape)
print(emb_Xt.shape)


(132, 10, 50)
(56, 10, 50)


In [59]:
from keras.layers import *
from keras.models import Sequential

In [66]:
model = Sequential()
model.add(LSTM(32,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(32,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 10, 32)              │          10,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 10, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             165 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 5)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,109 (74.64 KB)

 Trainable params: 19,109 (74.64 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - acc: 0.2009 - loss: 1.6180 - val_acc: 0.2857 - val_loss: 1.5940
Epoch 2/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - acc: 0.2131 - loss: 1.5887 - val_acc: 0.2857 - val_loss: 1.5894
Epoch 3/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - acc: 0.3108 - loss: 1.5607 - val_acc: 0.2857 - val_loss: 1.5824
Epoch 4/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - acc: 0.2871 - loss: 1.5439 - val_acc: 0.2857 - val_loss: 1.5822
Epoch 5/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - acc: 0.3666 - loss: 1.5044 - val_acc: 0.2857 - val_loss: 1.5817
Epoch 6/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - acc: 0.3145 - loss: 1.4895 - val_acc: 0.2857 - val_loss: 1.5873
Epoch 7/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - acc: 0.3335 - loss: 1.4645 - val_acc: 0.2857 - val_loss: 1.5825
Epoch 8/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - acc: 0.4221 - loss: 1.4157 - val_acc: 0.2857 - val_loss: 1.5796
Epoch 9/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - acc: 0.3874 - loss: 1.3777 -

In [68]:
model.evaluate(emb_Xt,Yt)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - acc: 0.6265 - loss: 1.3318


[1.2690249681472778, 0.6428571343421936]

In [69]:
pred = model.predict(emb_Xt)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step


In [70]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(pred[i]))]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😓
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😓
This girl is messing with me
😓
😓
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😓
Congratulation for having a baby
😁
😁
stop messing around
😓
⚾
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
😓
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😓
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
